In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime


In [2]:
driver = webdriver.Chrome()
driver.get("https://www.indycar.com/INDYNXT/Schedule")

# Wait for dynamic content to load if needed

sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('li', {'class': 'schedule-list__item'})

driver.quit()

                                        

In [3]:
# schedule_details

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuits = []
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year



month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12,
        'DEC': 12
    }





In [5]:
circuit_naming = {}
link_names = []
for i in schedule_details:
    link_temp = i.find('a', class_='panel-trigger schedule-list__title')
    link = link_temp['href'].split('/')[-1]
    link_names.append(link)
    
circ_names = ['St Petersburg Street Circuit' '\n' 'St. Petersburg, FL.',
              'Barber Motorsports Park' '\n' 'Birmingham, AL',
              'Indianapolis Motor Speedway Road Course' '\n' 'Indianapolis, IN.',
              'Indianapolis Motor Speedway Road Course' '\n' 'Indianapolis, IN.',
              'Downtown Detroit Street Circuit' '\n' 'Detroit, MI',
              'Road America' '\n' 'Elkhart Lake, WI.',
              'Laguna Seca Raceway' '\n' 'Monterey, CA',
              'Laguna Seca Raceway' '\n' 'Monterey, CA',
              'Mid-Ohio Sports Car Course' '\n' 'Lexington, OH',
              'Iowa Speedway' '\n' 'Newton, IA',
              'Gateway Motorsports Park' '\n' 'Madison, IL.',
              'Portland International Raceway' '\n' 'Portland, OR.',
              'Milwaukee Mile' '\n' 'West Allis, WI',
              'Nashville Superspeedway' '\n' 'Lebanon, TN'
             ]

circuit_naming = dict(zip(link_names, circ_names))

In [6]:
# #Testing##
# link_prefix = 'https://www.indycar.com'

# schedule_details[1].find('a', class_='panel-trigger schedule-list__title')
# link_temp = schedule_details[1].find('a', class_='panel-trigger schedule-list__title')
# link = link_temp['href']
# driver = webdriver.Chrome()
# driver.get(link_prefix + link)
# event_soup = BeautifulSoup(driver.page_source, 'html.parser')
# desc = event_soup.find('h2', {'style':''}).text.strip()
# desc

# # # <div class="race-list__date text">Sunday,   Sep 15 </div>

In [7]:
link_prefix = 'https://www.indycar.com'
round_counter = 1

for i in schedule_details:
    link_temp = i.find('a', class_='panel-trigger schedule-list__title')
    link = link_temp['href']
    driver = webdriver.Chrome()
    driver.get(link_prefix + link)
    event_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    desc = event_soup.find('h2', {'style':''}).text.strip()
    dist = event_soup.find('h2', {'class':'race-details'}).text.split('|')
    lap_count = dist[0].strip().replace('\n', '').replace('LAPS', 'Laps').split()
    lap_count = ' '.join(lap_count)  # Join the split elements to remove excess space
    distance = dist[1].strip().replace('\n', '')
    formatted_output = f"{lap_count}, {distance}"                  
    descs.append(desc + '\n' + formatted_output)
    
    temp_circuit = link_temp['href'].split('/')[-1]
    converted_circuit = circuit_naming[temp_circuit]
    circuits.append(converted_circuit)

    temp_title = 'IndyNXT Round ' + '0' + str(round_counter)
    round_counter += 1
    titles.append(temp_title)
    
    temp_date = event_soup.find('div', class_='race-list__date text').text.split(' ')
    temp_month = temp_date[-3]
    conv_month = month_mapping[temp_month]
    temp_day = temp_date[-2]
    months.append(conv_month)
    days.append(int(temp_day))
    
    temp_time = event_soup.find('div', class_='race-list__time text').text.split('-')
    if 'TBD' in temp_time:
        start_hours.append(12)
        start_minutes.append(0)
        end_hours.append(13)
        end_minutes.append(0)
        
    else:
        start_h, start_m = temp_time[0].split(':')[0], temp_time[0].split(':')[-1].strip(' PM').strip(' A')
        if 'PM' in temp_time[0]:
            if start_h == '12':
                start_hours.append(int(start_h))
            else:  
                start_h = int(start_h) + 12
                start_hours.append(start_h)
        else:
            start_hours.append(int(start_h))
        
        if start_m == '00':
            start_minutes.append(0)
        else:
            start_minutes.append(int(start_m))

        end_h, end_m = temp_time[1].split(':')[0], temp_time[1].split(':')[-1].strip(' PM ET').strip(' A')
        if 'PM' in temp_time[1]:
            if end_h == ' 12':
                end_hours.append(int(end_h))
            else:  
                end_h = int(end_h) + 12
                end_hours.append(end_h)
        else:
            end_hours.append(int(end_h))
            
        if end_m == '00':
            end_minutes.append(0)
        else:
            end_minutes.append(int(end_m))
     

In [8]:
temp_time

['12:50 PM ', ' 2:05 PM ET']

In [9]:
days 



[8, 26, 10, 11, 2, 9, 22, 23, 7, 13, 17, 25, 31, 15]

In [10]:
months 

[3, 4, 5, 5, 6, 6, 6, 6, 7, 7, 8, 8, 8, 9]

In [11]:
titles


['IndyNXT Round 01',
 'IndyNXT Round 02',
 'IndyNXT Round 03',
 'IndyNXT Round 04',
 'IndyNXT Round 05',
 'IndyNXT Round 06',
 'IndyNXT Round 07',
 'IndyNXT Round 08',
 'IndyNXT Round 09',
 'IndyNXT Round 010',
 'IndyNXT Round 011',
 'IndyNXT Round 012',
 'IndyNXT Round 013',
 'IndyNXT Round 014']

In [12]:
descs 


['Grand Prix of St. Petersburg\nStreets of St. Petersburg 45 Laps, 81 Miles',
 'Grand Prix of Alabama\nBarber Motorsports Park 35 Laps, 80.5 Miles',
 'Indianapolis Grand Prix Race 1\nIndianapolis Motor Speedway Road Course 35 Laps, 85.37 Miles',
 'Indianapolis Grand Prix Race 2\nIndianapolis Motor Speedway Road Course 35 Laps, 85.37 Miles',
 'Detroit Grand Prix\nStreets of Detroit 45 Laps, 74.03 Miles',
 'Grand Prix at Road America\nRoad America 20 Laps, 80.28 Miles',
 'Grand Prix of Monterey Race 1\nWeatherTech Raceway Laguna Seca 35 Laps, 78.33 Miles',
 'Grand Prix of Monterey Race 2\nWeatherTech Raceway Laguna Seca 35 Laps, 78.33 Miles',
 'Grand Prix at Mid-Ohio\nMid-Ohio Sports Car Course 35 Laps, 79.03 Miles',
 'Iowa 100\nIowa Speedway 75 Laps, 67.05 Miles',
 'OUTFRONT Showdown\nWorld Wide Technology Raceway 75 Laps, 93.75 Miles',
 'Grand Prix of Portland\nPortland International Raceway 35 Laps, 68.74 Miles',
 'Milwaukee 100\nMilwaukee Mile Laps, ',
 'Music City Grand Prix\nNashvi

In [13]:
circuits 


['St Petersburg Street Circuit\nSt. Petersburg, FL.',
 'Barber Motorsports Park\nBirmingham, AL',
 'Indianapolis Motor Speedway Road Course\nIndianapolis, IN.',
 'Indianapolis Motor Speedway Road Course\nIndianapolis, IN.',
 'Downtown Detroit Street Circuit\nDetroit, MI',
 'Road America\nElkhart Lake, WI.',
 'Laguna Seca Raceway\nMonterey, CA',
 'Laguna Seca Raceway\nMonterey, CA',
 'Mid-Ohio Sports Car Course\nLexington, OH',
 'Iowa Speedway\nNewton, IA',
 'Gateway Motorsports Park\nMadison, IL.',
 'Portland International Raceway\nPortland, OR.',
 'Milwaukee Mile\nWest Allis, WI',
 'Nashville Superspeedway\nLebanon, TN']

In [14]:
time 


[]

In [15]:
start_hours 


[13, 14, 11, 13, 10, 13, 15, 15, 11, 14, 15, 13, 15, 12]

In [16]:
start_minutes 


[35, 30, 5, 0, 20, 5, 25, 55, 15, 5, 55, 10, 50, 50]

In [17]:
end_hours


[14, 15, 11, 14, 23, 14, 16, 17, 12, 15, 17, 14, 17, 14]

In [18]:
end_minutes

[20, 15, 55, 15, 35, 20, 40, 10, 30, 20, 10, 25, 5, 5]

In [19]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [20]:
IndyNXT = Calendar()
IndyNXT.add('prodid', '-//My calendar product//example.com//')
IndyNXT.add('version', '2.0')
uid = 110

In [21]:
 
for title, circuit, day, month, desc, s_h, s_m, e_h, e_m in zip(titles, circuits, days, months, descs, start_hours, start_minutes, end_hours, end_minutes):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, s_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(year, month, day, e_h, e_m, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(2024, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    IndyNXT.add_component(ievent)

In [22]:
IndyNXT

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'IndyNXT Round 01''), 'DESCRIPTION': vText('b'Grand Prix of St. Petersburg\\nStreets of St. Petersburg 45 Laps\\, 81 Miles''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000002A0112C3EE0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000002A0112C39D0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000002A0112C3430>, 'LOCATION': vText('b'St Petersburg Street Circuit\\nSt. Petersburg\\, FL.''), 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'IndyNXT Round 02''), 'DESCRIPTION': vText('b'Grand Prix of Alabama\\nBarber Motorsports Park 35 Laps\\, 80.5 Miles''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x000002A0112C31F0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x000002A0112C30A0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x000002A0112C39A0>, 'LOCATION': vText('b'Barber Motorsports Park\\nBirmingham\\, AL''), 'UID': vText('b'111'')

In [23]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'IndyNXT.ics'), 'wb')
f.write(IndyNXT.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
